<a href="https://colab.research.google.com/github/andr3w1699/Intelligent_System_for_Pattern_Recognition/blob/main/Assignement3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Airline_Reviews.csv to Airline_Reviews.csv


In [24]:
import pandas as pd

df = pd.read_csv('Airline_Reviews.csv')
df.head()

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no


In [25]:
print(df['Overall_Rating'].unique())

['9' '1' '8' '2' '3' '5' '6' '7' '4' 'n']


In [26]:
print(df['Overall_Rating'].value_counts())

Overall_Rating
1    11595
2     2296
9     1768
8     1757
3     1356
7     1192
4      859
n      842
5      830
6      676
Name: count, dtype: int64


In [27]:
print(df['Overall_Rating'].dtype)

object


In [28]:
# Remove all rows where Overall_Rating is 'n'
df = df[df['Overall_Rating'] != 'n']

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

In [30]:
# 2. Basic cleaning
df.dropna(subset=['Review', 'Overall_Rating'], inplace=True)

# Remove rows with non-numeric Overall_Rating values
df = df[pd.to_numeric(df['Overall_Rating'], errors='coerce').notnull()]

# Concatenate Review_Title with Review at the beginning
df['Review'] = df['Review_Title'] + ' ' + df['Review']

# Now safely convert to int
df['Overall_Rating'] = df['Overall_Rating'].astype(int)

# Optional: limit to a valid range, e.g., 1 to 9
df = df[df['Overall_Rating'].between(1, 9)]

# Map Overall_Rating to Sentiment
def rating_to_sentiment(rating):
    if 1 <= rating <= 3:
        return 'Negative'
    elif 4 <= rating <= 6:
        return 'Neutral'
    elif 7 <= rating <= 9:
        return 'Positive'
    return 'Unknown'  # In case there are any outliers

# Apply the function to create the 'Sentiment' column
df['Sentiment'] = df['Overall_Rating'].apply(rating_to_sentiment)

# View the result
df[['Review', 'Overall_Rating', 'Sentiment']].head()


,Review,Overall_Rating,Sentiment
0,"""pretty decent airline"" Moroni to Moheli. Tu...",9,Positive
1,"""Not a good airline"" Moroni to Anjouan. It is...",1,Negative
2,"""flight was fortunately short"" Anjouan to Dz...",1,Negative
3,"""I will never fly again with Adria"" Please d...",1,Negative
4,"""it ruined our last days of holidays"" Do not ...",1,Negative


In [32]:
# 3. Tokenize and pad
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
padded = pad_sequences(sequences, maxlen=200, truncating='post')

# 4. Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Sentiment'])

# 5. Split data
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

# 6. Build the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 7. Train the model
history = model.fit(X_train, y_train, epochs=5, validation_split=0.1, batch_size=64, callbacks=[early_stopping])

# 8. Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


252/252 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.7051 - loss: 0.7602 - val_accuracy: 0.7717 - val_loss: 0.5885
Epoch 2/5
252/252 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7835 - loss: 0.5333 - val_accuracy: 0.7734 - val_loss: 0.5626
Epoch 3/5
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8048 - loss: 0.4811 - val_accuracy: 0.7717 - val_loss: 0.5960
Epoch 4/5
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.8311 - loss: 0.4163 - val_accuracy: 0.7521 - val_loss: 0.6204
Epoch 5/5
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8602 - loss: 0.3570 - val_accuracy: 0.7264 - val_loss: 0.6974
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7867 - loss: 0.5485
Test Accuracy: 0.7850


In [39]:
import numpy as np

# Load the GloVe embeddings (this example uses a 50-dimensional GloVe model)
# You can replace this URL with a different GloVe file if you prefer another version
!wget http://nlp.stanford.edu/data/glove.6B.zip

# Unzip the GloVe files
!unzip glove.6B.zip


--2025-05-04 18:20:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-04 18:20:24--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-04 18:20:25--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

KeyboardInterrupt: 

In [40]:
# Load the GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load GloVe embeddings (use glove.6B.50d.txt or glove.6B.100d.txt, etc.)
embedding_dim = 200  # or 100, 200, 300 based on the downloaded file
glove_path = f'glove.6B.{embedding_dim}d.txt'

embeddings_index = load_glove_embeddings(glove_path)
print(f'Found {len(embeddings_index)} word vectors.')

Found 400000 word vectors.


In [41]:
embedding_dim = 200  # Or 100, 200, 300 based on which GloVe file you used
vocab_size = len(tokenizer.word_index) + 1  # Plus 1 for padding token
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in tokenizer.word_index.items():
    if word in embeddings_index:
        embedding_matrix[index] = embeddings_index[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (36707, 200)


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=200,
              weights=[embedding_matrix], trainable=False),  # Set trainable=False to keep embeddings fixed
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1, batch_size=64, callbacks=[early_stopping] )

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.6702 - loss: 0.7982 - val_accuracy: 0.7420 - val_loss: 0.6391
Epoch 2/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.7400 - loss: 0.6286 - val_accuracy: 0.7750 - val_loss: 0.5741
Epoch 3/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7644 - loss: 0.5763 - val_accuracy: 0.7644 - val_loss: 0.5639
Epoch 4/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7655 - loss: 0.5620 - val_accuracy: 0.7812 - val_loss: 0.5530
Epoch 5/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.7658 - loss: 0.5455 - val_accuracy: 0.7795 - val_loss: 0.5407
Epoch 6/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.7819 - loss: 0.5108 - val_accuracy: 0.7823 - val_loss: 0.5354
Epoch 7/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7864 - loss: 0.5069 - val_accuracy: 0.7818 - val_loss: 0.5396
Epoch 8/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.7979 - loss: 0.4733 - val_ac